In [1]:
def reproduceResult():
  seed_value= 0

  
  with tf.device("/cpu:0"):
    ...


  os.environ['PYTHONHASHSEED']=str(seed_value)
  np.random.seed(0)
  rn.seed(0)


  session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, 
                                          inter_op_parallelism_threads=1)


  tf.compat.v1.set_random_seed(seed_value)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
  tf.compat.v1.keras.backend.set_session(sess)
  tf.compat.v1.keras.backend.clear_session()

  


In [2]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
  
import os 
import numpy as np
import tensorflow as tf
import random as rn
from tensorflow import keras

reproduceResult()
# %tensorflow_version 2.x
# import tensorflow as tf
# tf.test.gpu_device_name()
# from scipy import integrate
# import os
# import numpy as np
# from tensorflow import keras
import tempfile
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
# import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K


from tensorflow.keras import callbacks

from keras_lr_finder import LRFinder
from clr.clr_callback import CyclicLR

import tensorflow_model_optimization as tfmot



import keras_tuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from attention import Attention

C:\Users\moshi\AppData\Local\Temp/ipykernel_19964/628588012.py:43: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
df1 = pd.read_csv('qqp.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)

df2 = pd.read_csv('qqp.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)


C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df1.is_duplicate[df1.is_duplicate == 1]= "True"
df1.is_duplicate[df1.is_duplicate == 0]= "False"

df2.is_duplicate[df2.is_duplicate == 1]= "True"
df2.is_duplicate[df2.is_duplicate == 0]= "False"

C:\Users\moshi\AppData\Local\Temp/ipykernel_19964/3241943720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.is_duplicate[df1.is_duplicate == 1]= "True"
C:\Users\moshi\AppData\Local\Temp/ipykernel_19964/3241943720.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.is_duplicate[df1.is_duplicate == 0]= "False"
C:\Users\moshi\AppData\Local\Temp/ipykernel_19964/3241943720.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2

In [5]:
df1["is_duplicate"].value_counts()

False    229468
True     134378
Name: is_duplicate, dtype: int64

In [6]:
df2["is_duplicate"].value_counts()

False    229468
True     134378
Name: is_duplicate, dtype: int64

In [7]:
# taking acceptable and deleting unacceptable
df3 = df1[~df1.is_duplicate.str.contains('True')]

In [8]:
df3.head

<bound method NDFrame.head of             id    qid1    qid2  \
0       133273  213221  213222   
2       360472  364011  490273   
4       183004  279958  279959   
5       119056  193387  193388   
6       356863  422862   96457   
...        ...     ...     ...   
363836  290434  239873  411728   
363838  281756  401562  401563   
363840  343528  471663  471664   
363843  136211  217377  217378   
363844  302720  425744  285638   

                                                question1  \
0       How is the life of a math student? Could you d...   
2            What causes stool color to change to yellow?   
4       Where can I find a power outlet for my laptop ...   
5       How not to feel guilty since I am Muslim and I...   
6                          How is air traffic controlled?   
...                                                   ...   
363836  Which is the better way of celebrating Indepen...   
363838  What's it like to have an observant Muslim cof...   
363840      

In [9]:
# randomly picking 250 rows
df3 = df3.sample(n=250, replace=True)

In [10]:
df3["is_duplicate"].value_counts()

False    250
Name: is_duplicate, dtype: int64

In [11]:
# taking unacceptable and deleting unacceptable
df4 = df2[~df2.is_duplicate.str.contains("False")]

In [12]:
# randomly picking 250 rows
df4 = df4.sample(n=250, replace=True)

In [13]:
df4["is_duplicate"].value_counts()

True    250
Name: is_duplicate, dtype: int64

In [14]:
#concating acceptable and unacceptable
frames = [df3, df4]

temp = pd.concat(frames)

In [15]:
temp["is_duplicate"].value_counts()

False    250
True     250
Name: is_duplicate, dtype: int64

In [16]:
train, test = train_test_split(temp, test_size=0.2, stratify = temp['is_duplicate'], random_state = 42)
num_classes = 2
embed_num_dims = 300
max_seq_len = 50

x_train = train['question1'] + train['question2']
x_test = test['question1'] + test['question2']

y_train = train['is_duplicate']
y_test = test['is_duplicate']

texts_train = x_train
texts_test = x_test

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['question1'] + train['question2'])

sequence_train = tokenizer.texts_to_sequences(texts_train)
sequence_test = tokenizer.texts_to_sequences(texts_test)

index_of_words = tokenizer.word_index

vocab_size = len(index_of_words) + 1

print('Number of unique words: {}'.format(len(index_of_words)))

X_train_pad = pad_sequences(sequence_train, maxlen = max_seq_len, padding='pre' )
X_test_pad = pad_sequences(sequence_test, maxlen = max_seq_len,  padding='pre')

print(X_train_pad)


encoding = {
    'True': 1,
    'False': 0
}

y_train = [encoding[x] for x in train['is_duplicate']]
y_test = [encoding[x] for x in test['is_duplicate']]



y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

def create_embedding_matrix(word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    with open('C:/Users/moshi/Python Code/Untitled Folder/cc.en.300.vec',encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)
print(embedd_matrix.shape)


Number of unique words: 2019
[[   0    0    0 ... 1012   22   56]
 [   0    0    0 ...  526  527 1017]
 [ 195  133 1018 ...  369  531  102]
 ...
 [   0    0    0 ...    9  457  524]
 [   0    0    0 ...  131    1   16]
 [   0    0    0 ...  364 1010  525]]
(2020, 300)


# Random Search


In [17]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import  convert_variables_to_constants_v2_as_graph

def get_flops(model):
    concrete = tf.function(lambda inputs: model(inputs))
    concrete_func = concrete.get_concrete_function(
        [tf.TensorSpec([1, *inputs.shape[1:]]) for inputs in model.inputs])
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(concrete_func)
    with tf.Graph().as_default() as graph:
        tf.graph_util.import_graph_def(graph_def, name='')
        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        flops = tf.compat.v1.profiler.profile(graph=graph, run_meta=run_meta, cmd="op", options=opts)
        return flops.total_float_ops

In [18]:
from tensorflow.keras import regularizers
import time
LOG_DIR = f"{int(time.time())}"
seed_value= 0



def build_model(hp):
  
  reproduceResult()

  print('Ya it comes here')
  unit_attention = hp.Int("attention_unit",min_value =32, max_value = 128, step = 16)
  fake_val = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_unit = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn_1_dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_unit = hp.Int("lstm_unit",min_value =64, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)
  cnn_2_unit = hp.Int("cnn_2_unit",min_value =64, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn_2_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)



  seq_input = keras.layers.Input(shape=(max_seq_len,))

  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = max_seq_len,
                          weights = [embedd_matrix])(seq_input)

  cnn = keras.layers.Conv1D(cnn_1_unit,3,kernel_regularizer=regularizers.l2(1e-4),
                            bias_regularizer=regularizers.l2(1e-2),
                            activity_regularizer=regularizers.l2(1e-4))(embedded)
  cnn = keras.layers.Activation(activation='relu')(cnn)
  cnn = keras.layers.BatchNormalization()(cnn)
  cnn = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn)


  lstm = keras.layers.Bidirectional(keras.layers.LSTM(lstm_unit, recurrent_regularizer=regularizers.l2(1e-4),
                                                      return_sequences=True,kernel_regularizer=regularizers.l2(1e-4),
                                                      bias_regularizer=regularizers.l2(1e-2),
                                                      activity_regularizer=regularizers.l2(1e-4),input_shape =(48,)))(cnn)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)
  
  

  max_pooling = keras.layers.GlobalMaxPooling1D()(lstm)
  output = keras.layers.Dense(num_classes, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

  model.summary()
  
  print("The FLOPs is:{}".format(get_flops(model)) ,flush=True )
    
  print(" ")

  return model



clr_step_size = int((len(X_train_pad)/64))
base_lr = 1e-3
max_lr = 6e-3
mode = 'exp_range'


clr = CyclicLR(base_lr = base_lr, max_lr = max_lr, step_size = clr_step_size, mode = mode)

stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')

tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 40,
    executions_per_trial = 1,
    directory = LOG_DIR
    )
  
tuner.search(x=X_train_pad,y = y_train,epochs = 20, batch_size = 64,callbacks = [stop,clr], 
             validation_data = (X_test_pad,y_test))


tuner.results_summary()

Trial 40 Complete [00h 00m 07s]
val_accuracy: 0.5799999833106995

Best val_accuracy So Far: 0.699999988079071
Total elapsed time: 00h 09m 47s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in 1653408987\untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
attention_unit: 48
cnn_1_unit: 32
cnn_1_dropout: 0.2
lstm_unit: 96
lstm_dropout: 0.30000000000000004
cnn_2_unit: 224
cnn_2_dropout: 0.1
Score: 0.699999988079071
Trial summary
Hyperparameters:
attention_unit: 64
cnn_1_unit: 80
cnn_1_dropout: 0.2
lstm_unit: 256
lstm_dropout: 0.2
cnn_2_unit: 160
cnn_2_dropout: 0.2
Score: 0.6700000166893005
Trial summary
Hyperparameters:
attention_unit: 96
cnn_1_unit: 64
cnn_1_dropout: 0.1
lstm_unit: 64
lstm_dropout: 0.4
cnn_2_unit: 128
cnn_2_dropout: 0.4
Score: 0.6700000166893005
Trial summary
Hyperparameters:
attention_unit: 96
cnn_1_unit: 80
cnn_1_dropout: 0.1
lstm_unit: 192
lstm_dropout: 0.4
cnn_2_unit: 160
cnn_2_dropout:

In [19]:
tuner.search_space_summary()

Search space summary
Default search space size: 7
attention_unit (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': None}
cnn_1_unit (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 96, 'step': 16, 'sampling': None}
cnn_1_dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.3, 'step': 0.1, 'sampling': None}
lstm_unit (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 256, 'step': 32, 'sampling': None}
lstm_dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
cnn_2_unit (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 256, 'step': 32, 'sampling': None}
cnn_2_dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
